## Import dependencies

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Deep Learning
import tensorflow as tf
from tf_agents.environments import py_environment, tf_py_environment, suite_gym, suite_atari
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.drivers import dynamic_step_driver
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import trajectory

import warnings
warnings.filterwarnings('ignore')

## Read in initial CSV pulled from game data

In [2]:
# read in game data
rpsls_game_df = pd.read_csv('gamedata.csv')
print(len(rpsls_game_df))
rpsls_game_df.head(11)

473


,Game ID,Round,Computer Choice,Player Choice,Round Winner
0,589051,1,paper,scissors,Player
1,589051,2,paper,scissors,Player
2,589051,3,paper,rock,Computer
3,589051,4,scissors,rock,Player
4,589051,5,lizard,lizard,Draw
5,589051,6,rock,scissors,Computer
6,589051,7,paper,rock,Computer
7,589051,8,scissors,scissors,Draw
8,589051,9,spock,scissors,Computer
9,589051,10,rock,scissors,Computer


## Capture previous game player/computer move, round winnner, and encode these values

In [3]:
# create new columnns to capture the previous round's player move, computer move, and round winner
rpsls_game_df['Previous Player Move'] = rpsls_game_df.groupby('Game ID')['Player Choice'].shift(1)
rpsls_game_df['Previous Computer Move'] = rpsls_game_df.groupby('Game ID')['Computer Choice'].shift(1)
rpsls_game_df['Previous Round Winner'] = rpsls_game_df.groupby('Game ID')['Round Winner'].shift(1)

# Drop the rows with NaN values that now exist for the first round of each game
rpsls_game_df = rpsls_game_df.dropna().reset_index(drop=True)

from sklearn.preprocessing import LabelEncoder

# create a label encoder
le = LabelEncoder()

# List of columns to encode
columns_to_encode = ['Previous Player Move', 'Previous Computer Move', 'Previous Round Winner']

# Loop through the list and encode each column
for column in columns_to_encode:
    rpsls_game_df[f'{column} Encoded'] = le.fit_transform(rpsls_game_df[column])

rpsls_game_df.head(11)

,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0


## Normalize round numbers

In [4]:
# Normalize round number
scaler = MinMaxScaler()
rpsls_game_df['Normalized Round'] = scaler.fit_transform(rpsls_game_df[['Round']])

rpsls_game_df.head(11)

,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Normalized Round
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2,0.000
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2,0.125
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0,0.250
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2,0.375
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1,0.500
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0,0.625
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0,0.750
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1,0.875
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0,1.000
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0,0.000


# Create a win streak count column

In [5]:
# Define a function to calculate win/loss streaks
def calculate_streaks(data, winner_col):
    streaks = []
    streak = 0
    last_winner = None
    for winner in data[winner_col]:
        if winner == last_winner:
            streak += 1
        else:
            streak = 1
        streaks.append(streak)
        last_winner = winner
    return streaks

rpsls_game_df['Player Streak'] = calculate_streaks(rpsls_game_df, 'Round Winner')

rpsls_game_df.head(11)


,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Normalized Round,Player Streak
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2,0.000,1
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2,0.125,1
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0,0.250,1
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2,0.375,1
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1,0.500,1
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0,0.625,2
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0,0.750,1
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1,0.875,1
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0,1.000,2
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0,0.000,1


# Create state representation of each round as pertaining to previous round

In [6]:
# combine multiple features to create a state representation for each round
rpsls_game_df['State Representation'] = rpsls_game_df.apply(lambda x: f"{x['Previous Player Move']}-{x['Previous Computer Move']}-{x['Normalized Round']}", axis=1)

rpsls_game_df.head(11)


,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Normalized Round,Player Streak,State Representation
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2,0.000,1,scissors-paper-0.0
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2,0.125,1,scissors-paper-0.125
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0,0.250,1,rock-paper-0.25
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2,0.375,1,rock-scissors-0.375
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1,0.500,1,lizard-lizard-0.5
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0,0.625,2,scissors-rock-0.625
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0,0.750,1,rock-paper-0.75
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1,0.875,1,scissors-scissors-0.875
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0,1.000,2,scissors-spock-1.0
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0,0.000,1,spock-paper-0.0


In [7]:
# calculate the frequency of player choices following a loss
condition = (rpsls_game_df['Previous Round Winner'] == 'Computer')
player_choices_post_loss = rpsls_game_df[condition]['Player Choice'].value_counts(normalize=True)


## Capture whether or not player repeated their move

In [8]:
# calculate whether the player repeated their move
rpsls_game_df['Player Repeat Move'] = rpsls_game_df['Player Choice'] == rpsls_game_df['Previous Player Move']

rpsls_game_df.head(11)

,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Normalized Round,Player Streak,State Representation,Player Repeat Move
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2,0.000,1,scissors-paper-0.0,True
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2,0.125,1,scissors-paper-0.125,False
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0,0.250,1,rock-paper-0.25,True
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2,0.375,1,rock-scissors-0.375,False
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1,0.500,1,lizard-lizard-0.5,False
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0,0.625,2,scissors-rock-0.625,False
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0,0.750,1,rock-paper-0.75,False
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1,0.875,1,scissors-scissors-0.875,True
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0,1.000,2,scissors-spock-1.0,True
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0,0.000,1,spock-paper-0.0,False


## Encode each player/computer choice, and round winner

In [9]:
# finally, before running models, encode computer and player choice, and round winner

# create a label encoder
le = LabelEncoder()

# List of columns to encode
columns_to_encode = ['Computer Choice', 'Player Choice', 'Round Winner']

# Loop through the list and encode each column
for column in columns_to_encode:
    rpsls_game_df[f'{column} Encoded'] = le.fit_transform(rpsls_game_df[column])

rpsls_game_df.head(11)

,Game ID,Round,Computer Choice,Player Choice,Round Winner,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Normalized Round,Player Streak,State Representation,Player Repeat Move,Computer Choice Encoded,Player Choice Encoded,Round Winner Encoded
0,589051,2,paper,scissors,Player,scissors,paper,Player,3,1,2,0.000,1,scissors-paper-0.0,True,1,3,2
1,589051,3,paper,rock,Computer,scissors,paper,Player,3,1,2,0.125,1,scissors-paper-0.125,False,1,2,0
2,589051,4,scissors,rock,Player,rock,paper,Computer,2,1,0,0.250,1,rock-paper-0.25,True,3,2,2
3,589051,5,lizard,lizard,Draw,rock,scissors,Player,2,3,2,0.375,1,rock-scissors-0.375,False,0,0,1
4,589051,6,rock,scissors,Computer,lizard,lizard,Draw,0,0,1,0.500,1,lizard-lizard-0.5,False,2,3,0
5,589051,7,paper,rock,Computer,scissors,rock,Computer,3,2,0,0.625,2,scissors-rock-0.625,False,1,2,0
6,589051,8,scissors,scissors,Draw,rock,paper,Computer,2,1,0,0.750,1,rock-paper-0.75,False,3,3,1
7,589051,9,spock,scissors,Computer,scissors,scissors,Draw,3,3,1,0.875,1,scissors-scissors-0.875,True,4,3,0
8,589051,10,rock,scissors,Computer,scissors,spock,Computer,3,4,0,1.000,2,scissors-spock-1.0,True,2,3,0
9,743001,2,scissors,scissors,Draw,spock,paper,Computer,4,1,0,0.000,1,spock-paper-0.0,False,3,3,1


## Reorder columns

In [10]:
# Define the new column order
new_column_order = [
    'Game ID', 
    'Round', 
    'Normalized Round', 
    'Player Streak', 
    'State Representation', 
    'Player Choice', 
    'Computer Choice',
    'Round Winner',
    'Player Choice Encoded', 
    'Computer Choice Encoded', 
    'Round Winner Encoded', 
    'Previous Player Move', 
    'Previous Computer Move', 
    'Previous Round Winner', 
    'Previous Player Move Encoded',
    'Previous Computer Move Encoded',
    'Previous Round Winner Encoded',
    'Player Repeat Move'
]

# Reorder the columns
rpsls_game_df = rpsls_game_df[new_column_order]
rpsls_game_df.head(11)

,Game ID,Round,Normalized Round,Player Streak,State Representation,Player Choice,Computer Choice,Round Winner,Player Choice Encoded,Computer Choice Encoded,Round Winner Encoded,Previous Player Move,Previous Computer Move,Previous Round Winner,Previous Player Move Encoded,Previous Computer Move Encoded,Previous Round Winner Encoded,Player Repeat Move
0,589051,2,0.000,1,scissors-paper-0.0,scissors,paper,Player,3,1,2,scissors,paper,Player,3,1,2,True
1,589051,3,0.125,1,scissors-paper-0.125,rock,paper,Computer,2,1,0,scissors,paper,Player,3,1,2,False
2,589051,4,0.250,1,rock-paper-0.25,rock,scissors,Player,2,3,2,rock,paper,Computer,2,1,0,True
3,589051,5,0.375,1,rock-scissors-0.375,lizard,lizard,Draw,0,0,1,rock,scissors,Player,2,3,2,False
4,589051,6,0.500,1,lizard-lizard-0.5,scissors,rock,Computer,3,2,0,lizard,lizard,Draw,0,0,1,False
5,589051,7,0.625,2,scissors-rock-0.625,rock,paper,Computer,2,1,0,scissors,rock,Computer,3,2,0,False
6,589051,8,0.750,1,rock-paper-0.75,scissors,scissors,Draw,3,3,1,rock,paper,Computer,2,1,0,False
7,589051,9,0.875,1,scissors-scissors-0.875,scissors,spock,Computer,3,4,0,scissors,scissors,Draw,3,3,1,True
8,589051,10,1.000,2,scissors-spock-1.0,scissors,rock,Computer,3,2,0,scissors,spock,Computer,3,4,0,True
9,743001,2,0.000,1,spock-paper-0.0,scissors,scissors,Draw,3,3,1,spock,paper,Computer,4,1,0,False


### Verify length of initial dataframe

In [11]:
print(len(rpsls_game_df))

395


## Conduct data enrichment by oversampling (append dataframe to itself to increase data available to train)

In [12]:
# append dataframe to itself 5 additional times to increase amount of data for training
augmented_df = pd.concat([rpsls_game_df]*12, ignore_index=True)
print(len(augmented_df))

4740


## Run model with base dataset

In [13]:
# Assume you have a sequence of moves as a list of integers
move_sequence = rpsls_game_df['Player Choice Encoded'].tolist()  # This needs to be encoded as integers

# Split the sequence into a training set and a validation set
train_size = int(len(move_sequence) * 0.5)  # Use 50% of the data for training
train_sequence = move_sequence[:train_size]
val_sequence = move_sequence[train_size:]

# Create a dataset from the training sequence
train_dataset = tf.data.Dataset.from_tensor_slices(train_sequence)
train_dataset = train_dataset.window(size=5, shift=1, drop_remainder=True)
train_dataset = train_dataset.flat_map(lambda window: window.batch(5))
train_dataset = train_dataset.map(lambda window: (window[:-1], window[-1]))
train_dataset = train_dataset.batch(32).prefetch(1)

# Create a dataset from the validation sequence
val_dataset = tf.data.Dataset.from_tensor_slices(val_sequence)
val_dataset = val_dataset.window(size=5, shift=1, drop_remainder=True)
val_dataset = val_dataset.flat_map(lambda window: window.batch(5))
val_dataset = val_dataset.map(lambda window: (window[:-1], window[-1]))
val_dataset = val_dataset.batch(32).prefetch(1)

# Build an LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
])

# Compile the model with a smaller learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Fit the model with early stopping
model.fit(train_dataset, epochs=1000, validation_data=val_dataset)


Epoch 1/1000

7/7 [==============================] - 5s 160ms/step - loss: 2.5565 - accuracy: 0.0000e+00 - val_loss: 2.5126 - val_accuracy: 0.0000e+00
Epoch 2/1000
7/7 [==============================] - 0s 11ms/step - loss: 2.4818 - accuracy: 0.0000e+00 - val_loss: 2.4349 - val_accuracy: 0.0000e+00
Epoch 3/1000
7/7 [==============================] - 0s 11ms/step - loss: 2.4047 - accuracy: 0.0104 - val_loss: 2.3407 - val_accuracy: 0.1289
Epoch 4/1000
7/7 [==============================] - 0s 11ms/step - loss: 2.3085 - accuracy: 0.1399 - val_loss: 2.2354 - val_accuracy: 0.1701
Epoch 5/1000
7/7 [==============================] - 0s 11ms/step - loss: 2.2112 - accuracy: 0.1813 - val_loss: 2.1534 - val_accuracy: 0.2165
Epoch 6/1000
7/7 [==============================] - 0s 11ms/step - loss: 2.1430 - accuracy: 0.1917 - val_loss: 2.1052 - val_accuracy: 0.2165
Epoch 7/1000
7/7 [==============================] - 0s 13ms/step - loss: 2.1086 - accuracy: 0.1969 - val_loss: 2.0727 - val_accuracy: 0

## Run model with enriched dataset

In [14]:
# Assume you have a sequence of moves as a list of integers
move_sequence = augmented_df['Player Choice Encoded'].tolist()  # This needs to be encoded as integers

# Split the sequence into a training set and a validation set
train_size = int(len(move_sequence) * 0.5)  # Use 50% of the data for training
train_sequence = move_sequence[:train_size]
val_sequence = move_sequence[train_size:]

# Create a dataset from the training sequence
train_dataset = tf.data.Dataset.from_tensor_slices(train_sequence)
train_dataset = train_dataset.window(size=5, shift=1, drop_remainder=True)
train_dataset = train_dataset.flat_map(lambda window: window.batch(5))
train_dataset = train_dataset.map(lambda window: (window[:-1], window[-1]))
train_dataset = train_dataset.batch(32).prefetch(1)

# Create a dataset from the validation sequence
val_dataset = tf.data.Dataset.from_tensor_slices(val_sequence)
val_dataset = val_dataset.window(size=5, shift=1, drop_remainder=True)
val_dataset = val_dataset.flat_map(lambda window: window.batch(5))
val_dataset = val_dataset.map(lambda window: (window[:-1], window[-1]))
val_dataset = val_dataset.batch(32).prefetch(1)

# Build an LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
])

# Compile the model with a smaller learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Fit the model with early stopping
model.fit(train_dataset, epochs=1000, validation_data=val_dataset)

Epoch 1/1000
74/74 [==============================] - 5s 18ms/step - loss: 2.0206 - accuracy: 0.1653 - val_loss: 1.8229 - val_accuracy: 0.2054
Epoch 2/1000
74/74 [==============================] - 0s 7ms/step - loss: 1.7710 - accuracy: 0.2232 - val_loss: 1.7269 - val_accuracy: 0.2502
Epoch 3/1000
74/74 [==============================] - 0s 6ms/step - loss: 1.7030 - accuracy: 0.2502 - val_loss: 1.6825 - val_accuracy: 0.2502
Epoch 4/1000
74/74 [==============================] - 0s 7ms/step - loss: 1.6704 - accuracy: 0.2502 - val_loss: 1.6585 - val_accuracy: 0.2502
Epoch 5/1000
74/74 [==============================] - 0s 6ms/step - loss: 1.6514 - accuracy: 0.2502 - val_loss: 1.6443 - val_accuracy: 0.2502
Epoch 6/1000
74/74 [==============================] - 0s 7ms/step - loss: 1.6400 - accuracy: 0.2502 - val_loss: 1.6352 - val_accuracy: 0.2502
Epoch 7/1000
74/74 [==============================] - 1s 7ms/step - loss: 1.6324 - accuracy: 0.2502 - val_loss: 1.6289 - val_accuracy: 0.2502
Epoch

## Save neural network model as file to impor into RL model

In [15]:
# save deep learning model into file
model.save("neural_network_model.h5")